In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import desc
from nltk.tokenize import WordPunctTokenizer
import numpy as np
import pyspark.sql.functions as F
import tensorflow as tf
import tensorflow_hub as hub
import re
import nltk
#import matplotlib.pyplot as plt
from pyspark import SparkContext

VBox()

Starting Spark application


import all the functions and methods we need for the code.

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Assignment2") \
    .getOrCreate()

initialize the application

In [ ]:
dic = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
raw_data_df = spark.read.csv(dic,header=True,sep='\t')

#raw_data_df.show(2, False)

Read the tsv.gz file from s3 and split columns by "\t" with header left.
The quoted code is only for testing and checking the results.

In [ ]:
organized_data = raw_data_df.select(['customer_id','product_id','review_id'])
#organized_data.show()

Select the columns we need from all columns, for stage 1, we only need three columns: "customer_id","product_id", and "review_id".

In [ ]:
TotalReviewsNumber = organized_data.count()
#print(TotalReviewsNumber)

To count the total number of reviews

In [ ]:
data_group_by_customer = organized_data.groupBy('customer_id').count()
#TotalCustomerNumber = organized_data.select('customer_id').distinct().count()
TotalCustomerNumber = data_group_by_customer.count()
#print(TotalCustomerNumber)

group the reviews with customer and count the number of it, to get the total number of customer

In [ ]:
data_group_by_product = organized_data.groupBy('product_id').count()
TotalProductNumber = data_group_by_product.count()
print(TotalProductNumber)

group the reviews with product and count the number of it, to get the total number of product

In [ ]:
product_sort_by_reviews = data_group_by_product.sort(desc("count"))
#product_sort_by_reviews.show()

to get the top 10 and 1 product,we need to sort the results of groupBy(product)

In [ ]:
product_with_most_reviews = product_sort_by_reviews.filter("count>=5").limit(1)

In [ ]:
top_10_product_with_most_reviews = product_sort_by_reviews.filter("count>=5").limit(10)
# top_10_product_with_most_reviews.show()

get the top 10 product with most reviews

In [ ]:
sort_by_reviews = data_group_by_customer.sort(desc("count"))

In [ ]:
customer_with_most_reviews = sort_by_reviews.limit(1)
#customer_with_most_reviews.show()

get the top 1 customer with most reviews

In [ ]:
top_10_customer_with_most_reviews = sort_by_reviews.filter("count>=5").limit(10)
#top_10_customer_with_most_reviews.show()

In [ ]:
customer_median = sort_by_reviews.approxQuantile("count",[0.5],0)

calculate median of the customer with a descending result of reviews given by customers

In [ ]:
product_median = product_sort_by_reviews.approxQuantile("count",[0.5],0)

In [ ]:
# print(median)

this line is for checking the result of median.